In [3]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregar os dados
cleaned_eur_brl_file_path = "C:/Users/milen/OneDrive/Documentos/TCC/new/Merged_Final_Combined_EUR_BRL.csv"
cleaned_eur_brl_df = pd.read_csv(cleaned_eur_brl_file_path)

# Converter 'Data' para datetime e ordenar o dataframe por data
cleaned_eur_brl_df['Data'] = pd.to_datetime(cleaned_eur_brl_df['Data'])
cleaned_eur_brl_df.sort_values('Data', inplace=True)

# Corrigir o formato de 'Preco_Dolar', convertendo de string para float
cleaned_eur_brl_df['Preco_Dolar'] = cleaned_eur_brl_df['Preco_Dolar'].str.replace(',', '.').astype(float)

# Padronizar a coluna 'Preco_Dolar'
scaler = StandardScaler()
cleaned_eur_brl_df['Preco_Dolar_Scaled'] = scaler.fit_transform(cleaned_eur_brl_df[['Preco_Dolar']])

# Dividir os dados em treino e teste
X = cleaned_eur_brl_df.index.values.reshape(-1, 1)  # Usando o índice como recurso
y = cleaned_eur_brl_df['Preco_Dolar_Scaled'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)

# Padronizar os recursos de treino e teste
scaler_train = StandardScaler()
X_train_scaled = scaler_train.fit_transform(X_train)
X_test_scaled = scaler_train.transform(X_test)

# Criar e treinar o modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train_scaled, y_train)

# Fazer previsões no conjunto de teste
y_test_pred = rf_model.predict(X_test_scaled)

# Reverter a padronização para exibir os valores em sua escala original
y_test_pred_reversed = scaler.inverse_transform(y_test_pred.reshape(-1, 1)).flatten()
y_test_reversed = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Adicionar a coluna 'Data' ao DataFrame de previsões
eur_brl_test_prediction_df = pd.DataFrame({
    'Actual': y_test_reversed,
    'Predicted': y_test_pred_reversed,
    'Data': cleaned_eur_brl_df['Data'].values[-len(y_test):]
})

# Exibir as primeiras previsões e os valores reais
print(eur_brl_test_prediction_df.head())

#adicionar com e sem clima



   Actual  Predicted       Data
0  122.59   119.5823 2018-06-08
1  121.29   119.5823 2018-06-11
2  122.49   119.5823 2018-06-12
3  121.02   119.5823 2018-06-13
4  118.76   119.5823 2018-06-14


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Carregue seus dados aqui
data = pd.read_csv('"C:/Users/milen/OneDrive/Documentos/TCC/new/com_clima.csv')

# Preparação dos dados
# Substitua 'Preco_Dolar' pela sua variável alvo
features = data.drop('Preco_Dolar', axis=1)
target = data['Preco_Dolar']

# Convertendo variáveis categóricas em numéricas (se necessário)
label_encoders = {}
for column in features.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    features[column] = le.fit_transform(features[column])

# Divisão dos dados em treino e teste
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=42)

# Definição dos hiperparâmetros para a busca em grade
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Criando o modelo Random Forest para a busca em grade
rf = RandomForestRegressor(random_state=42)

# Criando o objeto Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Realizando a busca em grade
grid_search.fit(features_train, target_train)

# Melhores hiperparâmetros e pontuação
best_params = grid_search.best_params_
best_score = np.sqrt(-grid_search.best_score_)

# Exibindo os resultados
print("Melhores Hiperparâmetros:", best_params)
print("Melhor Score (RMSE):", best_score)
